In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
#!pip install dask[dataframe]
#!pip install pandas --upgrade
# We have to do this as dask is bad at guessing the types of objects for some reason
# Also, if you are having trouble with running the dataframe and continuously get 
# Fatal Python error: GC object already tracked, try to update pandas
data_types = {
    'ID':  np.int64,
    'Case Number':  object,
    'Date': object,
    'Block': object,
    'IUCR': object,
    'Primary Type': object,
    'Description':  object,
    'Location Description': object,
    'Arrest':  bool,
    'Domestic': bool,
    'Beat': np.int64,
    'District': np.float64,
    'Ward': np.float64,
    'Community Area': np.float64,
    'FBI Code': object,
    'X Coordinate': np.float64,
    'Y Coordinate': np.float64,
    'Year': np.int64,
    'Updated On': object,
    'Latitude': np.float64,
    'Longitude': np.float64,
    'Location': object
}
df = pd.read_csv('crimes.csv')
print("finished loading")

finished loading


c:\Anaconda2\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [167]:
import datetime
import locale

store = pd.HDFStore('store.h5')


data_subset = pd.DataFrame(df.head(2000).Date)

# add columns
data_subset.insert(0, 'Hour', 0)
data_subset.insert(0, 'Day', 0)
data_subset.insert(0, 'Month', 0)
data_subset.insert(0, 'Year', 0)

# convert string date time to 24 hour (just hour)
def to24Hour(x):
    date = datetime.datetime.strptime(x, '%x %X %p')
    hour = date.strftime('%H')
    am = x[-2:] == 'AM'
    if(am):
        if(hour == '12'):
            hour = '00'
        else:
            return hour
    else:
        if(hour != '12'):
            hour = str(int(hour) + 12)

    return hour


# convert date
locale.setlocale(locale.LC_ALL, '')
#datetime.datetime.strptime('06/25/2012 03:00:00 PM', '%x %X %p').strftime('%A')
#https://docs.python.org/2/library/datetime.html#strftime-strptime-behavior
data_subset['Hour'] = data_subset['Date'].apply(to24Hour)
data_subset['Day'] = data_subset['Date'].apply(lambda x: datetime.datetime.strptime(x, '%x %X %p').strftime('%w'))
data_subset['Month'] = data_subset['Date'].apply(lambda x: datetime.datetime.strptime(x, '%x %X %p').strftime('%m'))
data_subset['Year'] = data_subset['Date'].apply(lambda x: (datetime.datetime.strptime(x, '%x %X %p').strftime('%Y')))

# save to HDF5
store['data_subset'] = data_subset
#retrieve with varName = store['data_subset']

store.close()

data_subset



,Year,Month,Day,Hour,Date
0,2012,06,1,15,06/25/2012 03:00:00 PM
1,2012,06,0,22,06/24/2012 10:00:00 PM
2,2012,06,0,00,06/17/2012 12:01:00 AM
3,2012,06,1,15,06/25/2012 03:00:00 PM
4,2012,06,4,23,06/21/2012 11:00:00 PM
5,2012,06,1,17,06/25/2012 05:30:00 PM
6,2012,05,3,09,05/16/2012 09:00:00 AM
7,2012,06,1,18,06/25/2012 06:45:00 PM
8,2012,06,3,13,06/20/2012 01:00:00 PM
9,2012,06,5,13,06/22/2012 01:00:00 PM


In [85]:
import matplotlib.pyplot as plt

In [168]:
hourCounts = data_subset.Hour.value_counts().sort_index()

X = np.arange(len(hourCounts))

plt.bar(X, hourCounts.values)

plt.xticks(X, hourCounts.axes[0])

plt.show()

In [126]:
dayCounts = data_subset.Day.value_counts().sort_index()

X = np.arange(len(dayCounts))

plt.bar(X, dayCounts.values, align='center')

plt.xticks(X, dayCounts.axes[0]) # ['S', 'M', 'T', 'W', 'Th', 'F', 'Sa'])#

plt.show()

In [127]:
monthCounts = data_subset.Month.value_counts().sort_index()

X = np.arange(len(monthCounts))

plt.bar(X, monthCounts.values, align='center')

plt.xticks(X, monthCounts.axes[0])

plt.show()

In [ ]:
yearCounts = data_subset.Year.value_counts().sort_index()

X = np.arange(len(yearCounts))

plt.bar(X, yearCounts.values, align='center')

plt.xticks(X, yearCounts.axes[0])

plt.show()